In [20]:
import pandas as pd
import sys
import time
import pandas as pd
from sklearn import model_selection
from sklearn import ensemble, linear_model, svm, metrics, cluster, tree
from xgboost import XGBRegressor
from matplotlib import pyplot as plt
import scipy
import os
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.feature_selection import *
from sklearn.decomposition import PCA, NMF
from sklearn.model_selection import GridSearchCV, KFold

start_time = time.time()

n_cv = 5
n_jobs = 6
random_state = 3111696
file_path = "alldata.csv"

alldata = pd.read_csv(file_path, sep="\t")

data = alldata.iloc[:,8:].astype(float)
age = alldata["age"]

print(data.shape)
print(age.shape)

X_train, X_test, y_train, y_test = model_selection.train_test_split(data, age, 
                                                                    test_size=0.0, random_state=random_state)

print('Training started...')

pipe_regressor = Pipeline([
  ('regression', XGBRegressor(random_state=random_state))
])

parameters = [
    {   
        'regression__max_depth': [3, 4, 5],
        'regression__subsample': [0.6, 0.5, 0.4],
        'regression__colsample_bytree': [0.7, 0.6, 0.5],
        'regression__n_estimators': [100],
        #'regression__booster': ['gbtree', 'gblinear', 'dart'], \
        'regression__random_state': [random_state]
    }
]

optimized_regressor = GridSearchCV(pipe_regressor, parameters, \
                                       cv=KFold(n_splits=n_cv, shuffle=True, random_state=random_state), \
                                       error_score=0, scoring='r2', verbose=True, n_jobs=n_jobs, \
                                       pre_dispatch="1*n_jobs")

optimized_regressor.fit(X_train, y_train)
best_regressor = optimized_regressor.best_estimator_
best_result = optimized_regressor.cv_results_
print(optimized_regressor.best_params_)

best_score = optimized_regressor.best_score_
print("R2 score for training: %.2f" % best_score)

#prediction = best_regressor.predict(X_test)

#test_score = metrics.r2_score(y_test, prediction)
#print("Score for test: %.2f" % test_score)

print('Training finished')

end_time = time.time()
print("Time elapsed: %.2f" % (end_time - start_time))


(143, 27142)
(143,)
Training started...
Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  44 tasks      | elapsed:  7.9min
[Parallel(n_jobs=6)]: Done 135 out of 135 | elapsed: 21.0min finished
/home/anupkumar/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'regression__colsample_bytree': 0.7, 'regression__max_depth': 4, 'regression__n_estimators': 200, 'regression__random_state': 3111696, 'regression__subsample': 0.5}
R2 score for training: 0.65
Training finished
Time elapsed: 1303.51
